In [1]:
# import packages
from flask import Flask, request, jsonify
import pandas as pd
import numpy as np
import joblib

# Code below extract features from RNA-seq via NacSeq

In [2]:
from itertools import product

# define a function to read the RNA sequence based on specified window
def read_seq_by_chunksize(sample, window):
    return [sample[i:i+window] for i in range (0, len(sample)) if len(sample[i:i+window]) == window]

def vectorized_inputData(sample, combinations):
    return [sample.count(c) for c in combinations]

# extract the features
def extractFeature_via_nacSeq(inputData):
    
    print(inputData)
    
    # for RNA sequence only
    nucleotide = ['C', 'G', 'A', 'T']
    
    # perform cartesian-product based on k-value
    p_combinations_nac = sorted([''.join(str(s) for s in t) for t in product(nucleotide, repeat=1)])
    p_combinations_dac = sorted([''.join(str(s) for s in t) for t in product(nucleotide, repeat=2)])
    p_combinations_tac = sorted([''.join(str(s) for s in t) for t in product(nucleotide, repeat=3)])
    
    # encode the long-string input based on specified window
    encode_inputData_nac = list(map(read_seq_by_chunksize, inputData, [1]*inputData.shape[0]))
    encode_inputData_dac = list(map(read_seq_by_chunksize, inputData, [2]*inputData.shape[0]))
    encode_inputData_tac = list(map(read_seq_by_chunksize, inputData, [3]*inputData.shape[0]))
    
    # vectorized the output based on each combination found
    vectors_nac = list(map(vectorized_inputData, encode_inputData_nac, [p_combinations_nac]*len(encode_inputData_nac)))
    vectors_dac = list(map(vectorized_inputData, encode_inputData_dac, [p_combinations_dac]*len(encode_inputData_dac)))
    vectors_tac = list(map(vectorized_inputData, encode_inputData_tac, [p_combinations_tac]*len(encode_inputData_tac)))
    
    # convert to respective dataframe
    df_nac = pd.DataFrame(vectors_nac, columns=p_combinations_nac)
    df_dac = pd.DataFrame(vectors_dac, columns=p_combinations_dac)
    df_tac = pd.DataFrame(vectors_tac, columns=p_combinations_tac)
    
    # combine into single dataframe as input to predictive model
    data = pd.concat([df_nac, df_dac, df_tac], axis=1)
    
    return data

# Code below build a Flask Application for Model Api

In [3]:
# build Flask Application
app = Flask(__name__)

# load the machine learning model
# model = joblib.load("/Users/eesoonhang/Desktop/capstone_data/models/RFC_100trees/RFC_100trees_withRFE.pkl")
# rfe = joblib.load("/Users/eesoonhang/Desktop/capstone_data/models/RFC_100trees/RFE_33.pkl")

# define a predict function
@app.route('/sgcarsmart/recommend', methods=['POST'])
def recommend():
    data = request.get_json(force=True)
    return jsonify(data)

if __name__ == '__main__':
    app.run(port=5050, debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:9090/ (Press CTRL+C to quit)
 * Restarting with watchdog (fsevents)
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 15, in <module>
    from ipykernel import kernelapp as app
  File "/usr/local/anaconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 18, in <module>
    from IPython.core.application import (
  File "/usr/local/anaconda3/lib/python3.9/site-packages/IPython/__init__.py", line 56, in <module>
    from .terminal.embed import embed
  File "/usr/local/anaconda3/lib/python3.9/site-packages/IPython/terminal/embed.py", line 16, in <module>
    from IPython.terminal.interactiveshell import TerminalInteractiveShell
  File "/usr/local/anaconda3/lib/python3.9/site-packages/IPython/terminal/interactiveshell.py", line 35, in <module>
    from .debugger import TerminalPdb, Pdb
  File "/usr/local/anaconda3/lib/python3.9/site-packages/IPython/terminal/debugger.py", line 6, 

SystemExit: 1

/usr/local/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
